In [ ]:
from data_loader import *
from utils import *

In [ ]:
find_team("Lei")

In [ ]:
teams_data = pd.read_parquet(home_dir + "/data/Teams/team_league.parquet")

In [ ]:
unique_teams = teams_data['team_name'].unique()

In [ ]:
teams = unique_teams
season = 2023
visited_list = list()
for team in teams:
    try:
        dat = read_fixtures_for_season(team,season)
        visited_list.append(team)
    except:
        print(f"{team} does not have matches in {season}")

processing for Angers, 2023
Pulling for Angers,2023 with league_id: 61
Angers does not have matches in 2023
processing for Lille, 2023
Pulling for Lille,2023 with league_id: 61


/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:103: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fixture_dat_expanded['team_non_penalty_goals_scored'] = fixtures[(fixtures.fixture_id == fixture)]['goals_home'].values[0] - fixtures[(fixtures.fixture_id == fixture)]['score_penalty_home'].fillna(0).values[0]
/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:106: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fixture_dat_expanded['team_non_penalty_goals

processing for Lyon, 2023
Pulling for Lyon,2023 with league_id: 61


/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:103: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fixture_dat_expanded['team_non_penalty_goals_scored'] = fixtures[(fixtures.fixture_id == fixture)]['goals_home'].values[0] - fixtures[(fixtures.fixture_id == fixture)]['score_penalty_home'].fillna(0).values[0]
/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:106: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fixture_dat_expanded['team_non_penalty_goals

processing for Marseille, 2023
Pulling for Marseille,2023 with league_id: 61


...

processing for Torino, 2023
Pulling for Torino,2023 with league_id: 135


/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:103: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fixture_dat_expanded['team_non_penalty_goals_scored'] = fixtures[(fixtures.fixture_id == fixture)]['goals_home'].values[0] - fixtures[(fixtures.fixture_id == fixture)]['score_penalty_home'].fillna(0).values[0]
/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:106: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fixture_dat_expanded['team_non_penalty_goals

processing for Verona, 2023
Pulling for Verona,2023 with league_id: 135


/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:103: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fixture_dat_expanded['team_non_penalty_goals_scored'] = fixtures[(fixtures.fixture_id == fixture)]['goals_home'].values[0] - fixtures[(fixtures.fixture_id == fixture)]['score_penalty_home'].fillna(0).values[0]
/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:106: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fixture_dat_expanded['team_non_penalty_goals

Verona does not have matches in 2023
processing for Inter, 2023
Pulling for Inter,2023 with league_id: 135
Inter does not have matches in 2023
processing for Empoli, 2023
Pulling for Empoli,2023 with league_id: 135
Empoli does not have matches in 2023
processing for Salernitana, 2023
Pulling for Salernitana,2023 with league_id: 135
Salernitana does not have matches in 2023
processing for Spezia, 2023
Pulling for Spezia,2023 with league_id: 135
Spezia does not have matches in 2023
processing for Cremonese, 2023
Pulling for Cremonese,2023 with league_id: 135
Cremonese does not have matches in 2023
processing for Lecce, 2023
Pulling for Lecce,2023 with league_id: 135
Lecce does not have matches in 2023
processing for Monza, 2023
Pulling for Monza,2023 with league_id: 135
Monza does not have matches in 2023
processing for Ajax, 2023
Pulling for Ajax,2023 with league_id: 88
Ajax does not have matches in 2023
processing for Willem II, 2023
Pulling for Willem II,2023 with league_id: 88
Willem

In [56]:
teams = list(set(teams) - set(visited_list))
teams

['Metz',
 'Bahia',
 'PSV Eindhoven',
 'Bordeaux',
 'Verona',
 'Auxerre',
 'Avai',
 'Real Sociedad',
 'Saint Etienne',
 'Norwich',
 'Getafe',
 'Atletico Madrid',
 'FC Volendam',
 'Hamburger SV',
 'Real Betis',
 'NEC Nijmegen',
 'Barcelona',
 'Ajax',
 'Leeds',
 'AZ Alkmaar',
 'Rayo Vallecano',
 'Leicester',
 'MVV',
 'NAC Breda',
 'Celta Vigo',
 'Gremio',
 'Cambuur',
 'Inter',
 'Excelsior',
 'Monza',
 'Almeria',
 'Utrecht',
 'Angers',
 'Empoli',
 'Sport Recife',
 'Valladolid',
 'Fortuna Sittard',
 'Elche',
 'Borussia Mönchengladbach',
 'Espanyol',
 'Twente',
 'Lecce',
 'VfL Bochum',
 'Sevilla',
 'FC Eindhoven',
 'Salernitana',
 'Valencia',
 'Chapecoense-sc',
 'Estac Troyes',
 'Burnley',
 'Feyenoord',
 'Groningen',
 'Southampton',
 'Heerenveen',
 'GO Ahead Eagles',
 'Cadiz',
 'Cremonese',
 'Sampdoria',
 'Almere City FC',
 'Juventude',
 'Osasuna',
 'Mallorca',
 'Villarreal',
 'Real Madrid',
 'Ceara',
 'Waalwijk',
 'Spezia',
 'Emmen',
 'VVV Venlo',
 'Athletic Club',
 'FC Schalke 04',
 'Atlet

73